# Testing pipeline

In [ ]:
# PLIP


from plip.structure.preparation import PDBComplex
from pathlib import Path

def extract_plip_interaction_features(pdb_file):
    mol = PDBComplex()
    mol.load_pdb(str(pdb_file))  # use str in case it's a Path object
    mol.analyze()
    print(mol.ligands)

    '''
    for lig_id in complex.ligands:
        interactions = complex.interaction_sets[lig_id]
        features = {
            "hbonds_donor": len(interactions.hbonds_ldon),
            "hbonds_acceptor": len(interactions.hbonds_pdon),
            "hydrophobic_contacts": len(interactions.hydrophobic_contacts),
            "salt_bridges": len(interactions.saltbridges),
            "pi_stacking": len(interactions.pistacking),
            "pi_cation": len(interactions.pication),
            "metal_coordination": len(interactions.metal_complexes),
            "halogen_bonds": len(interactions.halogenbonds),
        }
    '''
    return features  # or collect multiple if needed



pdb_path = Path("/nvme2/helen/EnzymeStructuralFiltering/pipeline_output/superimposition/preparedfiles_for_superimposition/B9ZZP0_0_chai.pdb")
interaction_features = extract_plip_interaction_features(pdb_path)

if interaction_features:
    print(interaction_features)
else:
    print("No ligand or no interactions found.")


mol = PDBComplex()
mol.load_pdb("/nvme2/helen/EnzymeStructuralFiltering/pipeline_output/superimposition/preparedfiles_for_superimposition/B9ZZP0_0_chai.pdb")
mol.analyze()
print(mol.ligands)

longnames = [x.longname for x in mol.ligands]
bsids = [":".join([x.hetid, x.chain, str(x.position)]) for x in mol.ligands]
print(bsids)

indices = [j for j,x in enumerate(longnames) if x == 'LIG']

for idx in indices:
    bsid = bsids[idx]
    interactions = mol.interaction_sets[bsid]
    print(interactions.all_itypes)


Protein structure B9ZZP0_0_chai_Protein with ligands:
LIG:B:1


NameError: name 'features' is not defined

In [21]:
from plip.structure.preparation import PDBComplex

def extract_plip_interaction_features(pdb_path, defined_bsid=None, preferred_ligand_name="LIG"):
    '''
    Run PLIP on PDB structure and extract interaction features.
    '''
    mol = PDBComplex()
    mol.load_pdb(str(pdb_path))
    mol.analyze()

    # If present, use preferred_ligand_name
    if defined_bsid and defined_bsid in mol.interaction_sets:
        interactions = mol.interaction_sets[defined_bsid]
        return interactions

    # Fallback: Automatic ligand detection
    print("Defined BSID not found. Falling back to auto-detection.")

    longnames = [x.longname for x in mol.ligands]
    bsids = [":".join([x.hetid, x.chain, str(x.position)]) for x in mol.ligands]

    # Prefer ligands named 'LIG' (common for docked molecules)
    for idx, name in enumerate(longnames):
        if name == preferred_ligand_name:
            bsid = bsids[idx]
            if bsid in mol.interaction_sets:
                print(f"Auto-detected binding site: {bsid}")
                return mol.interaction_sets[bsid]

    # Fallback: just use the first available ligand
    if bsids:
        bsid = bsids[0]
        print(f"No preferred ligand found. Using first available: {bsid}")
        return mol.interaction_sets[bsid]

    print("No binding sites found.")
    return None


interactions = extract_plip_interaction_features("/nvme2/helen/EnzymeStructuralFiltering/pipeline_output/superimposition/preparedfiles_for_superimposition/B9ZZP0_0_chai.pdb", defined_bsid="LIG:B:1")

print(interactions.saltbridges)


AttributeError: 'PLInteraction' object has no attribute 'saltbridges'

In [23]:
print(list(mol.interaction_sets.keys()))


['LIG:B:1']


In [1]:
import pandas as pd
from plip.structure.preparation import PDBComplex

def extract_interaction_row(interactions, ligand_id):
    return {
        "ligand_id": ligand_id,
        "hbonds_donor": interactions.hbonds_ldon,
        "hbonds_acceptor": len(interactions.hbonds_pdon),
        ""
        "hydrophobic_contacts": (interactions.hydrophobic_contacts),
        "salt_bridges": len(interactions.saltbridge_lneg),
        "salt_bridges": len(interactions.all_hbonds_ldon),
        "pi_stacking": len(interactions.pistacking),
        #"pi_cation": len(interactions.pication),
        "metal_coordination": len(interactions.metal_complexes),
        #"halogen_bonds": len(interactions.halogenbonds),

    }

def get_all_interactions_as_df(pdb_file, preferred_ligand_name="LIG"):
    mol = PDBComplex()
    mol.load_pdb(str(pdb_file))
    mol.analyze()

    rows = []
    for lig in mol.ligands:
        lig_id = ":".join([lig.hetid, lig.chain, str(lig.position)])
        if lig_id not in mol.interaction_sets:
            continue
        interactions = mol.interaction_sets[lig_id]
        row = extract_interaction_row(interactions, lig_id)
        rows.append(row)

    return pd.DataFrame(rows)


df = get_all_interactions_as_df("pipeline_output/superimposition/preparedfiles_for_superimposition/B9ZZP0_0_chai.pdb")
df



,ligand_id,hbonds_donor,hbonds_acceptor,hydrophobic_contacts,salt_bridges,pi_stacking,metal_coordination
0,LIG:B:1,[],0,"[(Atom: 6 (-3.13 -9.07 2.13), 121, Atom: 6 (-6...",0,1,0


In [4]:
from plip.structure.preparation import PDBComplex

mol = PDBComplex()
mol.load_pdb("/nvme2/helen/EnzymeStructuralFiltering/pipeline_output/superimposition/preparedfiles_for_superimposition/B9ZZP0_0_chai.pdb")
mol.analyze()
print(mol.ligands)

longnames = [x.longname for x in mol.ligands]
bsids = [":".join([x.hetid, x.chain, str(x.position)]) for x in mol.ligands]
print(bsids)

indices = [j for j,x in enumerate(longnames) if x == 'LIG']

for idx in indices:
    bsid = bsids[idx]
    interactions = mol.interaction_sets[bsid]
    print(interactions.all_itypes)

[ligand(mol=<openbabel.pybel.Molecule object at 0x7f58840d9a10>, hetid='LIG', chain='B', position=1, water=[], members=[('LIG', 'B', 1)], longname='LIG', type='SMALLMOLECULE', atomorder=[1, 2, 3, 4, 5, 27, 28, 6, 7, 8, 26, 9, 10, 11, 12, 13, 14, 15, 25, 16, 17, 18, 23, 24, 19, 20, 21, 22], can_to_pdb={1: 3063, 2: 3064, 3: 3065, 4: 3066, 5: 3067, 27: 3068, 28: 3069, 6: 3070, 7: 3071, 8: 3072, 26: 3073, 9: 3074, 10: 3075, 11: 3076, 12: 3077, 13: 3078, 14: 3079, 15: 3080, 25: 3081, 16: 3082, 17: 3083, 18: 3084, 23: 3085, 24: 3086, 19: 3087, 20: 3088, 21: 3089, 22: 3090})]
['LIG:B:1']
[pistack(proteinring=aromatic_ring(atoms=[<openbabel.pybel.Atom object at 0x7f5883516090>, <openbabel.pybel.Atom object at 0x7f5883516110>, <openbabel.pybel.Atom object at 0x7f5883516190>, <openbabel.pybel.Atom object at 0x7f5883516210>, <openbabel.pybel.Atom object at 0x7f5883516290>, <openbabel.pybel.Atom object at 0x7f5883516310>], orig_atoms=[<openbabel.pybel.Atom object at 0x7f58833a00d0>, <openbabel.pyb

In [ ]:
# To Do: Calculate total stabilization energy using Francesca's code!

In [16]:
import pandas as pd

df = pd.read_pickle('/nvme2/helen/EnzymeStructuralFiltering/pipeline_output/geometricfiltering/LigandSASA.pkl')
df

,Entry,tool,best_structure,avg_rmsd,Squidly_CR_Position,distance_ligand_to_squidly_residues,distance_ligand_to_closest_nuc,Bürgi–Dunitz_angles_to_squidly_residues,Bürgi–Dunitz_angle_to_closest_nucleophile,ASvolume_dir,...,fpocket_Local_hydrophobic_density_Score,fpocket_Num_apolar_alpha_sphere,fpocket_Proportion_apolar_alpha_sphere,total_sasa,polar_sasa,apolar_sasa,sasa_ligand_in_complex,sasa_ligand_alone,buried_sasa,percentage_buried_sasa
0,B9ZZP0,boltz,B9ZZP0_model_3_boltz,2.611848,113|317|358,"{'SER_114': 4.175716225990459, 'ASP_318': 8.58...",{'SER_114': 4.175716225990459},"{'SER_114': nan, 'HIS_359': nan}",{'SER_114': nan},pipeline_output/geometricfiltering/ASVolume/B9...,...,53.8987,79,0.7900,53.979,3.259,50.720,4.284087,632.701273,628.417186,99.322889
1,B9ZZP0,chai,B9ZZP0_0_chai,3.289808,113|317|358,"{'SER_114': 6.869945778534208, 'ASP_318': 9.26...",{'CYS_296': 5.992924244473645},"{'SER_114': nan, 'HIS_359': nan}",{'CYS_296': nan},pipeline_output/geometricfiltering/ASVolume/B9...,...,58.0208,96,0.7805,144.493,22.523,121.970,20.916066,608.961042,588.044976,96.565287
2,B9ZZP0,vina,B9ZZP0_2_vina,3.885612,113|317|358,"{'SER_114': 3.96161961828745, 'ASP_318': 6.048...",{'SER_114': 3.96161961828745},"{'SER_114': nan, 'HIS_359': nan}",{'SER_114': nan},pipeline_output/geometricfiltering/ASVolume/B9...,...,17.2727,22,1.0000,10.869,0.000,10.869,2.495795,645.595318,643.099523,99.613412
3,C8WTQ0,boltz,C8WTQ0_model_0_boltz,1.585844,38|204,{'THR_205': 10.918059809325099},{'SER_155': 7.237408997700765},NaN,{'SER_155': nan},pipeline_output/geometricfiltering/ASVolume/C8...,...,76.3738,107,0.7926,134.556,25.870,108.687,63.374166,621.351620,557.977454,89.800595
4,C8WTQ0,chai,C8WTQ0_3_chai,1.411301,38|204,{'THR_205': 9.202288465376427},{'SER_155': 10.752609218231639},NaN,{'SER_155': nan},pipeline_output/geometricfiltering/ASVolume/C8...,...,73.9615,104,0.7536,182.335,52.723,129.612,95.290943,619.694141,524.403198,84.622907
5,C8WTQ0,vina,C8WTQ0_5_vina,2.199167,38|204,{'THR_205': 4.1367133088963275},{'CYS_186': 14.414936038706518},NaN,{'CYS_186': nan},pipeline_output/geometricfiltering/ASVolume/C8...,...,24.0000,53,0.3786,131.832,36.430,95.403,34.291880,605.805791,571.513911,94.339460
